In [1]:
import rosbag, sys, csv
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:

# get files
__curr_wd__ = os.path.realpath(os.getcwd())
__DOCG_file__ = os.path.realpath((os.getcwd())) + '/' + 'DOCGObjectTracks2.csv'
__OF_file__ = os.path.realpath(os.getcwd()) + '/' + 'fusedTracks2.csv'

with open(__DOCG_file__, 'r') as docg_file:
    docg_objects = list(csv.reader(docg_file, delimiter = ','))
    
with open(__OF_file__, 'r') as of_file:
    of_objects = list(csv.reader(of_file, delimiter = ','))

docg_objects = docg_objects[1:] #remove first row, that is header
docg_objects = np.array(docg_objects).astype("float") # convert all features to float

of_objects = of_objects[1:] # remove the first row that is header
of_objects = np.array(of_objects).astype("float") # convert all fatures to float

print(docg_objects.shape)
print(of_objects.shape)
# print(of_objects[1,:])
# print(of_objects[79048,:])


(79049, 9)
(79049, 9)


In [3]:
# create True class data
# the data in of and docg objects have same ids so they are True class and matches
true_class_data = np.concatenate([docg_objects, of_objects], axis =1 ) # concatenate the data
col_append = np.zeros(shape=(79049,1)) # create classification column

true_class_data = np.append(true_class_data,col_append, 1)

for row in true_class_data: # for each row
    if row[0] == row[9]: # if the ids match then make the classification as 1
        row[18] = 1
print(true_class_data[:,18])   # test by printing the column, the 18th column has the classification

print(true_class_data.shape)

[1. 1. 1. ... 1. 1. 1.]
(79049, 19)


In [4]:
# create False class data
# roll of_objects move first half OF data to last and last half of data to first
of_objects_roll = np.roll(of_objects, 39524, axis = 0)
false_class_data = np.concatenate([docg_objects, of_objects_roll], axis = 1) # concat the data
class_col_append = np.zeros(shape=(79049, 1)) # create classification column
false_class_data = np.append(false_class_data, class_col_append, 1)
print(false_class_data[:,18])
print(false_class_data.shape)
# x = np.arange(10)
# x = np.reshape(x,(5,2))
# print(x)
# x = np.roll(x,2, axis = 0)
# print(x)

[0. 0. 0. ... 0. 0. 0.]
(79049, 19)


In [52]:
#Prepare the data for ML
#delete the skewed data
true_class_Xy = true_class_data
false_class_Xy = false_class_data
true_class_Xy = np.delete(true_class_Xy, [0,9], 1)
false_class_Xy = np.delete(false_class_Xy, [0,9], 1)

Xy = np.concatenate([true_class_Xy, false_class_Xy], axis = 0)

#for i in range(20):
#    Xy
    
np.random.shuffle(Xy)
# from sklearn.model_selection import train_test_split
X = Xy[:,0:16]
y = Xy[:,16]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
X_train = X_train[0:2000,:]
y_train = y_train[0:2000]
print(y_train.shape)
X_test = X_test[2100:65000,:]
y_test = y_test[2100:65000]
# true_class_X = true_class_Xy[:]
#delete the unwanted columns
#arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
#print(arr)
#arr = np.delete(arr, 1, 1)
#print(arr)

(94858, 16)
(94858,)
(63240, 16)
(63240,)
(2000,)


In [53]:
# Neural Network MLP classifier
from sklearn.neural_network import MLPClassifier
NNclf = MLPClassifier(random_state = 0, hidden_layer_sizes=(20), activation = 'relu')
NNclf.fit(X_train, y_train)
print(NNclf.score(X_test, y_test))
y_pred = NNclf.predict(X_test)
count = 0
Ncount = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        count = count+1
    else:
        Ncount = Ncount+1
print(count)
print(Ncount)


0.987144259077527
60354
786


In [28]:
# RandomForest classifier
from sklearn.ensemble import RandomForestClassifier as rfc
clfr = rfc(n_estimators=120)
clfr.fit(X_train, y_train)
print(clfr.score(X_test,y_test))
#print(clfr.feature_importances_)
y_pred = clfr.predict(X_test)
print(y_pred)
count = 0
Ncount = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        count = count+1
    else:
        Ncount = Ncount+1
print(count)
print(Ncount)



0.9886706948640483
[0. 0. 1. ... 1. 0. 0.]
13090
150


In [24]:
## KNN Classifier
from sklearn.neighbors import KNeighborsClassifier
KNNclf = KNeighborsClassifier(n_neighbors=1)
KNNclf.fit(X_train, y_train)
print(KNNclf.score(X_test, y_test))
y_pred = KNNclf.predict(X_test)
count = 0
Ncount = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        count = count+1
    else:
        Ncount = Ncount+1
print(count)
print(Ncount)

0.9668
9668
332


In [25]:
#Lets test with SVM algorithm
from sklearn import svm
clf = svm.SVC(probability=True)
clf.fit(X_train, y_train)
print(clf.score(X_test,y_test))
y_pred = clf.predict(X_test)
print(y_pred)
count = 0
Ncount = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        count = count+1
    else:
        Ncount = Ncount+1
print(count)
print(Ncount)

0.9221
[0. 0. 1. ... 1. 0. 0.]
9221
779


In [10]:
# DecisionTrees
from sklearn import tree
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)
print(DTclf.score(X_test, y_test))
y_pred = DTclf.predict(X_test)
count = 0
Ncount = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        count = count+1
    else:
        Ncount = Ncount+1
print(count)
print(Ncount)

0.8775
1755
245


In [11]:
#NAiveBayes Classifier
from sklearn.naive_bayes import GaussianNB
NBclf = GaussianNB()
NBclf.fit(X_train, y_train)
print(NBclf.score(X_test, y_test))
y_pred = NBclf.predict(X_test)
count = 0
Ncount = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        count = count+1
    else:
        Ncount = Ncount+1
print(count)
print(Ncount)


0.4975
995
1005


In [55]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten, MaxPooling2D
from keras.models import Model, Sequential


siamese_net = Model(input=X_train, output=y_train)

/home/kjg9xn/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=array, inputs=array)`
  


TypeError: unhashable type: 'numpy.ndarray'